In [1]:
import dataAck
import portfolio
allTickers = dataAck.getAllTickersPlain()
import params

In [3]:
while True:
    import random
    tickerToTrade = allTickers[random.randint(0, len(allTickers)) - 1]
    print(tickerToTrade)
    
    tData = dataAck.getTrainingData(tickerToTrade)
    joinedData = None
    validTickers = None

    
    
    if tData is None:
        dataAck.logModel("Cache", {
            "type":"miss",
            "ticker":tickerToTrade,
            "day":str(portfolio.getToday())
        })

        tickersToPull = dataAck.getDataSourcesForTicker(tickerToTrade)
        print(tickersToPull)

        pulledData, validTickers = dataAck.downloadTickerData(tickersToPull)

        joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])
        
        dataAck.storeTrainingData(tickerToTrade, (joinedData, validTickers))
    else:
        joinedData = tData[0]
        validTickers = tData[1]
        dataAck.logModel("Cache", {
            "type":"hit",
            "ticker":tickerToTrade,
            "day":str(portfolio.getToday())
        })
        
    
    sManager = dataAck.seriesManager(validTickers)
    print(sManager.describe())

    import time
    import warnings
    import numpy as np
    warnings.filterwarnings("ignore")
    ##GET ALGOS INITIALLY GOOD
    runsSeen = 0
    while True:
        s = sManager.createSeries()
        while s.checkValidity(s.transformJoinedData(joinedData[:"2016-01-01"])) == False:
            s = sManager.createSeries()
            

        try:
            for defaultWindowSize in [5, 10, 22, 44]:
                for trees in [25, 50, 100, 150, 300]:
                    for predictionLength in [3, 5, 7, 10]:
                        if random.uniform(0,1) < 0.5: ##RANDOMLY SKIP
                            continue
                        b = dataAck.algoBlob(s, defaultWindowSize, trees, predictionLength, tickerToTrade)
                        algoReturn, factorReturn, predictions =  b.makePredictions(portfolio.prepareDataForModel(b, joinedData), daysToCheck = None, earlyStop = True)
                        if algoReturn is None:
                            toLog = factorReturn
                            if np.isnan(toLog["sharpe"]) == True:
                                raise ValueError('SHARPE IS NAN SO FAULTY SERIES')
                                
                            toLog["modelDescription"] = str(b.describe())
                            dataAck.logModel("Model Stopped Early", toLog)
                            print("Model Stopped Early", toLog)
                            continue
                        metrics = dataAck.vizResults(algoReturn[:-252], factorReturn[:-252], False)
                        print("TRAIN:", metrics)
                        if np.isnan(metrics["SHARPE"]) == True:
                            raise ValueError('SHARPE IS NAN SO FAULTY SERIES')
                        
                        if metrics["SHARPE"] > 1.0 and metrics["ACTIVITY"] > 0.7 and metrics["BETA"] < 0.6 and metrics["TREYNOR"] > 0.0:
                            ##STORE
                            testMetrics = dataAck.vizResults(algoReturn[-252:], factorReturn[-252:], False)
                            print("TEST:", testMetrics)
                            print("TODAY:", b.makeTodayPrediction(portfolio.prepareDataForModel(b, joinedData)))
                            dataAck.storeModelData(b, algoReturn, predictions)
                            dataAck.storeModel(b, metrics, testMetrics)
                        else:
                            dataAck.logModel("Model Skipped", {
                                "modelDescription":str(b.describe()),
                                "sharpe":metrics["SHARPE"], ##OVERLOADED IN FAIL
                                "activity":metrics["ACTIVITY"],
                                "beta":metrics["BETA"]
                            })
        except:
            print("FAILED", s.describe())
            dataAck.logModel("Series Failed", {
                "seriesDescription":str(s.describe())
            })

        runsSeen += 1

        if runsSeen > 10:
            ##START NEW TICKER
            dataAck.logModel("Search Update", {
                "message":"restarting search with different ticker",
                "currentTicker":tickerToTrade
            })
            break


XLP
ATTEMPTING PULL XLP
['AGG', 'EWC', 'EWH', 'EWJ', 'EWU', 'FXE', 'GDX', 'GLD', 'IAU', 'IEF', 'LQD', 'MDY', 'QQQ', 'SHY', 'SLV', 'TIP', 'TLT', 'USO', 'XLE', 'XLK', 'XLU', 'XLV', 'XLP']
GDX
FXE
IAU
GLD
AGG
EWU
EWC
EWJ
EWH
LQD
IEF
SHY
QQQ
SLV
TIP
MDY
USO
TLT
XLE
XLK
XLV
XLU
XLP
STORING XLP
['AGG', 'EWC', 'EWH', 'EWJ', 'EWU', 'FXE', 'GDX', 'GLD', 'IAU', 'IEF', 'LQD', 'MDY', 'QQQ', 'SHY', 'SLV', 'TIP', 'TLT', 'USO', 'XLE', 'XLK', 'XLU', 'XLV', 'XLP']
(5, ('SHY', 28, None, None, 2), 3, 'XLP')
SERIES (5, ('SHY', 28, None, None, 2), 3, 'XLP') WINDOW 5 TREES 25
THREAD  0 PROGRESS: 0.21739130434782608
THREAD  1 PROGRESS: 0.21739130434782608
THREAD  3 PROGRESS: 0.21739130434782608
THREAD  4 PROGRESS: 0.21739130434782608
THREAD  5 PROGRESS: 0.21739130434782608
THREAD  6 PROGRESS: 0.21739130434782608
THREAD  2 PROGRESS: 0.21739130434782608
THREAD  7 PROGRESS: 0.21739130434782608
THREAD  9 PROGRESS: 0.21739130434782608
THREAD  10 PROGRESS: 0.21739130434782608
THREAD  8 PROGRESS: 0.2173913043478260

Process ForkProcess-118:
Process ForkProcess-107:
Process ForkProcess-109:
Process ForkProcess-112:
Process ForkProcess-121:
Process ForkProcess-113:
Process ForkProcess-111:
Process ForkProcess-114:
Process ForkProcess-117:
Process ForkProcess-110:
Process ForkProcess-108:
Process ForkProcess-116:
Process ForkProcess-115:
Process ForkProcess-119:
Process ForkProcess-120:
Process ForkProcess-122:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/w

FAILED ('SHY', 28, None, None, 2)
(5, ('SHY', 32, None, 14, 2), 3, 'XLP')
SERIES (5, ('SHY', 32, None, 14, 2), 3, 'XLP') WINDOW 5 TREES 25
FAILED ('SHY', 32, None, 14, 2)
(5, ('GDX', 5, 8, 22, 3), 5, 'XLP')
SERIES (5, ('GDX', 5, 8, 22, 3), 5, 'XLP') WINDOW 5 TREES 25
FAILED ('GDX', 5, 8, 22, 3)
(5, ('EWC', 29, 20, None, 3), 3, 'XLP')
SERIES (5, ('EWC', 29, 20, None, 3), 3, 'XLP') WINDOW 5 TREES 25
FAILED ('EWC', 29, 20, None, 3)
(5, ('XLP', 30, 14, None, 1), 3, 'XLP')
SERIES (5, ('XLP', 30, 14, None, 1), 3, 'XLP') WINDOW 5 TREES 25
FAILED ('XLP', 30, 14, None, 1)
(5, ('SHY', 38, 20, None, 1), 7, 'XLP')
SERIES (5, ('SHY', 38, 20, None, 1), 7, 'XLP') WINDOW 5 TREES 25


Process ForkProcess-153:
Process ForkProcess-143:
Process ForkProcess-142:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/prohb125/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/prohb125/walkforwardTrader/dataAck.py", line 309, in runDayChunking
    pred = self.runDay(xVals[:int(i)], yVals[:int(i)], xVals[int(i)+44])
  File "/home/prohb125/anaconda3/lib/python3

FAILED ('SHY', 38, 20, None, 1)
(5, ('AGG', 7, 13, None, 3), 3, 'XLP')
SERIES (5, ('AGG', 7, 13, None, 3), 3, 'XLP') WINDOW 5 TREES 25
FAILED ('AGG', 7, 13, None, 3)
(5, ('TIP', 27, None, None, 3), 3, 'XLP')
SERIES (5, ('TIP', 27, None, None, 3), 3, 'XLP') WINDOW 5 TREES 25
FAILED ('TIP', 27, None, None, 3)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

